In [1]:
# Dependencies and setup
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
import sqlite3

In [2]:
# Load the JSON file into a Pandas DataFrame
json_file_path = Path("Data/picnic-settings.json")
raw_data_df = pd.read_json(json_file_path)

# Display sample data
raw_data_df.head()

,geo_point_2d,geo_shape,amenitytype,facility,projection,suburb_name,postcode,reserve_name,easting,northing,longitude,latitude,ward,googlemaps_drive_to
0,"{'lon': 145.3674000613, 'lat': -38.0979526786}","{'type': 'Feature', 'geometry': {'coordinates'...",None,Picnic Setting,MGA2020 zone 55,CLYDE NORTH,3978.0,None,356848,5782058,145.367400,-38.097953,EDRINGTON,https://www.google.com.au/maps/dir//-38.097952...
1,"{'lon': 145.3319956691, 'lat': -38.1246675609}","{'type': 'Feature', 'geometry': {'coordinates'...",None,Picnic Setting,MGA2020 zone 55,CLYDE,3978.0,Eliston Reserve,353796,5779038,145.331996,-38.124668,BALLA BALLA,https://www.google.com.au/maps/dir//-38.124667...
2,"{'lon': 145.2472688974, 'lat': -37.9921088616}","{'type': 'Feature', 'geometry': {'coordinates'...",Bench Timber Slats on Steel Frame,Picnic Setting,MGA2020 zone 55,DOVETON,3177.0,BADEN POWELL RESERVE,346092,5793611,145.247269,-37.992109,RIVER GUM,https://www.google.com.au/maps/dir//-37.992108...
3,"{'lon': 145.2646218828, 'lat': -37.9973288639}","{'type': 'Feature', 'geometry': {'coordinates'...",Bench Recycled Plastic,Picnic Setting,MGA2020 zone 55,HALLAM,3803.0,GUNN'S ROAD RESERVE,347627,5793060,145.264622,-37.997329,RIVER GUM,https://www.google.com.au/maps/dir//-37.997328...
4,"{'lon': 145.2641608835, 'lat': -37.9974518639}","{'type': 'Feature', 'geometry': {'coordinates'...",Bench Recycled Plastic,Picnic Setting,MGA2020 zone 55,HALLAM,3803.0,GUNN'S ROAD RESERVE,347586,5793046,145.264161,-37.997452,RIVER GUM,https://www.google.com.au/maps/dir//-37.997451...


In [3]:
# Drop unwanted columns
clean_data_df = raw_data_df.drop(["geo_point_2d", "geo_shape", "projection", "easting", "northing", "postcode", "facility"],axis='columns',inplace=False)

# Rename columns
clean_data_df.rename(columns={"amenitytype": "Amenity Type", "suburb_name": "Suburb Name", "reserve_name": "Reserve Name", "longitude": "Longitude", "latitude": "Latitude", "ward": "Ward", "googlemaps_drive_to": "Google Maps"}, inplace=True)

# View clean data
print(clean_data_df.to_string())

                           Amenity Type         Suburb Name                       Reserve Name   Longitude   Latitude         Ward                                                                                                      Google Maps
0                                  None         CLYDE NORTH                               None  145.367400 -38.097953    EDRINGTON   https://www.google.com.au/maps/dir//-38.09795267857918,145.36740006125757/@your+location,17z/data=!4m2!4m1!3e0
1                                  None               CLYDE                    Eliston Reserve  145.331996 -38.124668  BALLA BALLA    https://www.google.com.au/maps/dir//-38.12466756085634,145.3319956691417/@your+location,17z/data=!4m2!4m1!3e0
2     Bench Timber Slats on Steel Frame             DOVETON               BADEN POWELL RESERVE  145.247269 -37.992109    RIVER GUM  https://www.google.com.au/maps/dir//-37.992108861605686,145.24726889735484/@your+location,17z/data=!4m2!4m1!3e0
3                Bench R

In [4]:
# View the DataFrame information
clean_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 661 entries, 0 to 660
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Amenity Type  521 non-null    object 
 1   Suburb Name   660 non-null    object 
 2   Reserve Name  538 non-null    object 
 3   Longitude     661 non-null    float64
 4   Latitude      661 non-null    float64
 5   Ward          661 non-null    object 
 6   Google Maps   661 non-null    object 
dtypes: float64(2), object(5)
memory usage: 36.3+ KB


The following online resource was referred to when creating the SQLite Database:
https://www.fullstackpython.com/blog/export-pandas-dataframes-sqlite-sqlalchemy.html

In [5]:
# Create the connection engine for the database (.db format chosen)
engine = create_engine('sqlite:///Picnic_Database.db', echo=True)
sqlite_connection = engine.connect()

In [6]:
# Label the table
sqlite_table = "Picnic_Data"

# Move the data into a SQLite database
clean_data_df.to_sql(sqlite_table, sqlite_connection, if_exists="replace")

2023-06-20 18:39:04,038 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Picnic_Data")
2023-06-20 18:39:04,040 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-20 18:39:04,041 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Picnic_Data")
2023-06-20 18:39:04,042 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-20 18:39:04,044 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Picnic_Data" (
	"index" BIGINT, 
	"Amenity Type" TEXT, 
	"Suburb Name" TEXT, 
	"Reserve Name" TEXT, 
	"Longitude" FLOAT, 
	"Latitude" FLOAT, 
	"Ward" TEXT, 
	"Google Maps" TEXT
)


2023-06-20 18:39:04,045 INFO sqlalchemy.engine.Engine [no key 0.00092s] ()
2023-06-20 18:39:04,055 INFO sqlalchemy.engine.Engine COMMIT
2023-06-20 18:39:04,056 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Picnic_Data_index" ON "Picnic_Data" ("index")
2023-06-20 18:39:04,057 INFO sqlalchemy.engine.Engine [no key 0.00109s] ()
2023-06-20 18:39:04,064 INFO sqlalchemy.engine.Engine COMMIT
2023-06-20 18:39:04,073 INFO sqlalchemy.eng

661

In [7]:
# Close the connection to the database
sqlite_connection.close()

In [8]:
# Create the connection engine for the database (.db format chosen)
# con = create_engine(url, client_encoding='utf8')
# con.execute('alter table my_table add primary key(id)')

engine = create_engine('sqlite:///Picnic_Database.db', echo=True)
sqlite_connection = engine.connect()

# sqlite_connection.execute("ALTER TABLE Picnic_Data ADD PRIMARY KEY (index)")

sqlite_connection.execute("CREATE TABLE Picnic_Spots (id INTEGER PRIMARY KEY, Amenity Type TEXT, Suburb Name TEXT, Reserve Name TEXT, Longitude REAL, Latitude REAL, Ward TEXT, Google TEXT)")

sqlite_connection.execute("INSERT INTO Picnic_Spots SELECT * FROM Picnic_Data")

sqlite_connection.execute("DROP TABLE Picnic_Data")

sqlite_connection.close()


2023-06-20 18:39:04,145 INFO sqlalchemy.engine.Engine CREATE TABLE Picnic_Spots (id INTEGER PRIMARY KEY, Amenity Type TEXT, Suburb Name TEXT, Reserve Name TEXT, Longitude REAL, Latitude REAL, Ward TEXT, Google TEXT)
2023-06-20 18:39:04,145 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-20 18:39:04,152 INFO sqlalchemy.engine.Engine COMMIT
2023-06-20 18:39:04,153 INFO sqlalchemy.engine.Engine INSERT INTO Picnic_Spots SELECT * FROM Picnic_Data
2023-06-20 18:39:04,154 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-20 18:39:04,156 INFO sqlalchemy.engine.Engine COMMIT
2023-06-20 18:39:04,162 INFO sqlalchemy.engine.Engine DROP TABLE Picnic_Data
2023-06-20 18:39:04,163 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-20 18:39:04,169 INFO sqlalchemy.engine.Engine COMMIT


In [9]:
engine = create_engine("sqlite:///Picnic_Database.db")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables

Base.prepare(autoload_with=engine)

print(Base.classes.keys())

# picnic_places = Base.classes.Picnic_Data

['Picnic_Spots']
